In [ ]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import solve_ivp

In [ ]:
# declaring values

#######################################################################################################################################################################################
L = 5                          # length in m
B = 0.5                        # breadth in m
H = 0.5                        # height in m
E = 7E10                       # modulus of elasticity in N/m^4
D = 2700                       # density in Kg/m^3 
#######################################################################################################################################################################################

A = B*H                        # area in m^2
I = (B*H**3)/12                # area moment of inertia in m^4

In [ ]:
def make_csv(data, name):
    df = pd.DataFrame(data)
    name += '.csv'  
    df.to_csv(name, index = None, header = None)

def make_stiffness_matrix(n):
    k_matrix = np.array([
        [12*E*I/(L/n)**3, 6*E*I/(L/n)**2, -12*E*I/(L/n)**3, 6*E*I/(L/n)**2],
        [6*E*I/(L/n)**2, 4*E*I/(L/n), -6*E*I/(L/n)**2, 2*E*I/(L/n)],
        [-12*E*I/(L/n)**3, -6*E*I/(L/n)**2, 12*E*I/(L/n)**3, -6*E*I/(L/n)**2],
        [6*E*I/(L/n)**2, 2*E*I/(L/n), -6*E*I/(L/n)**2, 4*E*I/(L/n)]
    ])
    
    stiffness_matrix = np.zeros((2*n + 2, 2*n + 2), dtype = 'float')
    i, j = 0, 0
    counti, countj = 0, 0

    while i < 2*n + 2:
        while j < 2*n + 2:
            stiffness_matrix[i][j] += k_matrix[counti][countj]
            j += 1
            countj += 1
            if countj == 4:
                if counti != 3:
                    i += 1
                    countj = 0
                    j -= 4
                    counti += 1
                    continue
                if counti == 3:
                    if i == 2*n + 1:
                        break
                    j -= 2
                    i -= 1
                    counti, countj = 0, 0
        break
    return stiffness_matrix
    
def make_mass_matrix(n):
    m_matrix = np.array([
       [13*A*(L/n)*D/35, 11*A*((L/n)**2)*D/210, 9*A*(L/n)*D/70, -13*A*((L/n)**2)*D/420],
       [11*A*((L/n)**2)*D/210, A*((L/n)**3)*D/105, 13*A*((L/n)**2)*D/420, -A*((L/n)**3)*D/140],
       [9*A*(L/n)*D/70, 13*A*((L/n)**2)*D/420, 13*A*(L/n)*D/35, -11*A*((L/n)**2)*D/210],
       [-13*A*((L/n)**2)*D/420, -A*((L/n)**3)*D/140, -11*A*((L/n)**2)*D/210, A*((L/n)**3)*D/105]
    ])
    
    mass_matrix = np.zeros((2*n + 2, 2*n + 2), dtype = 'float')
    i, j = 0, 0
    counti, countj = 0, 0

    while i < 2*n + 2:
        while j < 2*n + 2:
            mass_matrix[i][j] += m_matrix[counti][countj]
            j += 1
            countj += 1
            if countj == 4:
                if counti != 3:
                    i += 1
                    countj = 0
                    j -= 4
                    counti += 1
                    continue
                if counti == 3:
                    if i == 2*n + 1:
                        break
                    j -= 2
                    i -= 1
                    counti, countj = 0, 0
        break
    return mass_matrix


In [ ]:
# CANTILEVER BEAM

N = 10                          # number of elements taken

STIFFNESS_MATRIX = make_stiffness_matrix(N)
# make_csv(STIFFNESS_MATRIX, "K")

MASS_MATRIX = make_mass_matrix(N)
# make_csv(MASS_MATRIX, "M")


STIFFNESS_MATRIX = np.delete(STIFFNESS_MATRIX, np.s_[0:2], 0)
STIFFNESS_MATRIX = np.delete(STIFFNESS_MATRIX, np.s_[0:2], 1)

MASS_MATRIX = np.delete(MASS_MATRIX, np.s_[0:2], 0)
MASS_MATRIX = np.delete(MASS_MATRIX, np.s_[0:2], 1)


evals, evecs = la.eigh(STIFFNESS_MATRIX, MASS_MATRIX)

# make_csv(evals, "evals")
# make_csv(evecs, "evecs")

frequencies = np.sqrt(evals)/(2*np.pi)
print(frequencies)



In [ ]:
# MARIX
M = MASS_MATRIX
K = STIFFNESS_MATRIX
C = np.zeros((2*N, 2*N))
A = np.vstack([
    np.hstack([
        np.zeros((2*N, 2*N)), 
        np.eye(2*N, 2*N)
    ]), 
    np.hstack([
        -np.matmul(np.linalg.inv(M), K), 
        -np.matmul(np.linalg.inv(M), C)
    ])
])

B = np.vstack([
    np.zeros((2*N, 2*N)), 
    np.linalg.inv(M)
])

Spring =400
Gap = 0.0002

Stiff = np.zeros(4*N)
Stiff[2*N - 1] = Spring

Functions = np.zeros((2*N, 1))

In [ ]:
def F(t, y):

    ########################################################################################################################################################################################
    Functions[-2] = 100*np.sin(2*np.pi*16.45*t)
    ########################################################################################################################################################################################

    yvec = np.array([[y[i] for i in range(4*N)]]).T

    return np.matmul(A, yvec) + np.matmul(B, Functions)

#######################################################################################################################################################################################
start_time = 0
end_time = 1
delta_t = 0.001
initial_conditions = np.zeros(4*N)
#######################################################################################################################################################################################

TE = np.arange(start_time, end_time, delta_t)

time_interval = np.array([start_time, end_time])

sol = solve_ivp(F, time_interval, initial_conditions, t_eval = TE, vectorized=True) #

T = sol.t
Y = sol.y

In [ ]:
# for i in range(len(Y)):
#     plt.figure(figsize=(20, 9))
#     plt.plot(T, Y[i], linewidth = 0.9, color = 'black')
#     plt.title('Y['+ str(i)+']')
#     plt.xlabel('Time')
#     plt.show() 

In [ ]:
# Vertical Displacement at the end
plt.figure(figsize=(20, 9))
plt.plot(T, Y[2*N - 2], linewidth = 0.8, color = 'black')
plt.title('Displacement at end')
plt.xlabel('Time')
plt.show()

In [ ]:
out_off_range = []
in_range = []

for i in Y[2*N - 2]:
    if i >= Gap or i <= -Gap:
        out_off_range.append(i)
    else:
        in_range.append(i)

In [ ]:
def F(t, y):

    ########################################################################################################################################################################################
    Functions[-2] = 100*np.sin(2*np.pi*16.45*t)
    ########################################################################################################################################################################################

    yvec = np.array([[y[i] for i in range(4*N)]]).T

    #
    #if y >= Gap or y <= -Gap:
    #    return np.matmul(Stiff, yvec) + np.matmul(A, yvec) + np.matmul(B, Functions)
    #else:

    # if Y[2*N - 2][-1] >= Gap or Y[2*N - 2][-1] <= -Gap:
    #     
    if t in out_off_range:
        return np.matmul(Stiff, yvec) + np.matmul(A, yvec) + np.matmul(B, Functions)
    else:
        return np.matmul(A, yvec) + np.matmul(B, Functions)

#######################################################################################################################################################################################
start_time = 0
end_time = 1
delta_t = 0.001
initial_conditions = np.zeros(4*N)
#######################################################################################################################################################################################

TE = np.arange(start_time, end_time, delta_t)

time_interval = np.array([start_time, end_time])

sol2 = solve_ivp(F, time_interval, initial_conditions, t_eval = TE, vectorized=True) #

TT = sol.t
YY = sol.y

In [ ]:
# Vertical Displacement at the end
plt.figure(figsize=(20, 9))
plt.plot(TT, YY[2*N - 2], linewidth = 0.8, color = 'black')
plt.title('Displacement at end')
plt.xlabel('Time')
plt.show() 